In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

In [3]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [4]:
from divers import plot_confusion_matrix

In [5]:
import sys
sys.path.append('./sompyLatmos/')     # Chemin du package
from sompy import SOMFactory
from sompy import SOMData

### Chargement et mise en forme des données

Chargement des différents jeux de représentation

In [6]:
# les imagettes 
x = np.loadtxt("./donnees/x.txt")
x_comp_names = [ "x["+str(i)+"]" for i in range(x.shape[0])]
#
# différentes projections des imagettes
hx = np.loadtxt("./donnees/hx.txt")
hx_comp_names = [ "hx["+str(i)+"]" for i in range(hx.shape[0])]
hy_comp_names = [ "hy["+str(i)+"]" for i in range(hx.shape[0])]
#
hx_hy = np.loadtxt("./donnees/hx_hy.txt")
hx_hy_comp_names = []
hx_hy_comp_names.extend(hx_comp_names)
hx_hy_comp_names.extend(hy_comp_names)
#
pb_comp_names = [ "pb["+str(i)+"]" for i in range(hx.shape[0])]
ph_comp_names = [ "ph["+str(i)+"]" for i in range(hx.shape[0])]
pb_ph = np.loadtxt("./donnees/pb_ph.txt")
pb_ph_comp_names = []
pb_ph_comp_names.extend(pb_comp_names)
pb_ph_comp_names.extend(ph_comp_names)
#
pg_comp_names = [ "pg["+str(i)+"]" for i in range(hx.shape[0])]
pd_comp_names = [ "pd["+str(i)+"]" for i in range(hx.shape[0])]
pg_pd = np.loadtxt("./donnees/pg_pd.txt")
pg_pd_comp_names = []
pg_pd_comp_names.extend(pg_comp_names)
pg_pd_comp_names.extend(pd_comp_names)
#
# combinaisons de projections
hx_hy_pb_ph = np.loadtxt("./donnees/hx_hy_pb_ph.txt")
hx_hy_pb_ph_comp_names = []
hx_hy_pb_ph_comp_names.extend(hx_hy_comp_names)
hx_hy_pb_ph_comp_names.extend(pb_ph_comp_names)
#
hx_hy_pg_pd = np.loadtxt("./donnees/hx_hy_pg_pd.txt")
hx_hy_pg_pd_comp_names = []
hx_hy_pg_pd_comp_names.extend(hx_hy_comp_names)
hx_hy_pg_pd_comp_names.extend(pg_pd_comp_names)


**Choix d'une représentation des données** 

In [7]:
#strChoix = 'x' problematique en théorie et en pratique
#strChoix = 'hx'
#strChoix = 'hx_hy'
#strChoix = 'pb_ph'
#strChoix = 'pg_pd'
#strChoix = 'hx_hy_pb_ph'
strChoix = 'hx_hy_pg_pd'
if strChoix == 'x':
    data = x.T
    comp_names = x_comp_names
elif strChoix == 'hx':
    data = hx.T
    comp_names = hx_comp_names
elif strChoix == 'hx_hy':
    data = hx_hy.T
    comp_names = hx_hy_comp_names
elif strChoix == 'pb_ph':
    data = pb_ph.T
    comp_names = pb_ph_comp_names
elif strChoix == 'pg_pd':
    data = pg_pd.T
    comp_names = pg_pd_comp_names
elif strChoix == 'hx_hy_pb_ph':
    data = hx_hy_pb_ph.T
    comp_names = hx_hy_pb_ph_comp_names
elif strChoix == 'hx_hy_pg_pd':
    data = hx_hy_pg_pd.T
    comp_names = hx_hy_pg_pd_comp_names
else:
    raise ValueError('choix inattendu')
masque=np.ones(len(comp_names))
del x, hx, hx_hy, pb_ph, pg_pd, hx_hy_pb_ph, hx_hy_pg_pd


Ici on a aussi les étiquettes correspondant à chacune des données. On les utilisera par la suite pour voir les performances de **l'algorithme qui est on le rappelle non supervisé**.   
On se servira de ces etiquettes pour évaluer la qualité du "résumé numérique" fourni par la carte topologique.

In [8]:
t = np.loadtxt("./donnees/t.txt").T
t[np.where(t==-1)] = 0
t_label_num = np.where(t==1)[1]
classnames = ['zero','un','deux','trois','quatre','cinq','six','sept','huit','neuf']
reverse_classnames =  {'zero':0,'un':1,'deux':2,'trois':3,'quatre':4,'cinq':5,'six':6,'sept':7,'huit':8,'neuf':9,'Nan':-1}
data_labels = [ classnames[i] for i in t_label_num]
data_labels =  np.array(data_labels)

Ici, on va mettre quelques données de coté pour évaluer les performances de l'apprentissage. Notamment pour évaluer la capacité de la partition réalisée par la carte en terme de classification des chiffres manuscrits.
Là encore, on rappelle que l'algorithme est non supervisé. Les étiquettes '`t`) ne sont pas prises en compte. Lors de l'apprentissage. La partition de la carte sera basée uniquement sur les données apprises (`X`). 
On évaluera la capacité de la carte à exploiter l'information fournie dans `X`.

In [9]:
N = 34*10
# rescale the data, use the traditional train/test split
#data, data_ = data[:N,:], data[N:,:]
#t_label_num, t_label_num_ = t_label_num[:N], t_label_num[N:]
#data_labels,data_labels_ = np.array(data_labels[:N]), np.array(data_labels[N:])
#data.shape

In [10]:
from sklearn.model_selection import train_test_split

data_dev, data_test, data_labels_dev, data_labels_test = train_test_split(data, data_labels, test_size=0.15, random_state=42)
data_train, data_val, data_labels_train, data_labels_val = train_test_split(data_dev, data_labels_dev, test_size=0.15, random_state=42)


**Création d'un objet sData**

In [11]:
sData_train = SOMData(data_train,      # les données 
                comp_names,# les noms des variables
                data_labels_train)

**Affichage des T-SNE**

In [12]:
#sData.plot_tsne()
import itertools

epochs= np.arange(10,120,25)
tmp_phase1=np.arange(2,10,2)
tmp_phase2=np.arange(0.3,3,0.4)



### Création de la carte et apprentissage

#### Création d'une carte topologique (objet sm)

In [13]:
#mapsize=(10,1)
mapsize=(12,12)
#mapsize=(10,10)
#mapsize=(3,7)
sm = SOMFactory().build(sData_train, 
                        mapsize=mapsize,
                        normalization = None, 
                        #initialization='random', # obligatoire pour x
                        initialization='pca',  
                        radius_train ='linear', 
                        name='Des Chiffres',
                        #lattice='rect',
                        lattice='hexa',
                        mask=masque,
                        components_to_plot=((0,1),(0,2),(1,2)))

#### Entraînement de la carte

In [14]:
'''
sm.train(n_job=1, 
         verbose=None, 
         train_rough_len=30, 
         train_rough_radiusin=3,
         train_rough_radiusfin=1,
         train_finetune_len=30,
         train_finetune_radiusin=2.5,
         train_finetune_radiusfin=0.25,
         watch_evolution = False)'''

'\nsm.train(n_job=1, \n         verbose=None, \n         train_rough_len=30, \n         train_rough_radiusin=3,\n         train_rough_radiusfin=1,\n         train_finetune_len=30,\n         train_finetune_radiusin=2.5,\n         train_finetune_radiusfin=0.25,\n         watch_evolution = False)'

In [15]:
#np.where(data_labels_estim[:,None]==np.array(classnames)[None,:])
#data_labels_estim

In [17]:
#np.where(data_labels_estim[:,None]==np.array(classnames_)[None,:])[1]  
#np.array([reverse_classnames[x] for x in data_labels_estim]).shape

In [21]:
list(combinations)

[(10, 2, 0.7),
 (10, 2, 1.0999999999999999),
 (10, 2, 1.5),
 (10, 2, 1.9),
 (10, 2, 2.3),
 (10, 2, 2.6999999999999997),
 (10, 4, 0.3),
 (10, 4, 0.7),
 (10, 4, 1.0999999999999999),
 (10, 4, 1.5),
 (10, 4, 1.9),
 (10, 4, 2.3),
 (10, 4, 2.6999999999999997),
 (10, 6, 0.3),
 (10, 6, 0.7),
 (10, 6, 1.0999999999999999),
 (10, 6, 1.5),
 (10, 6, 1.9),
 (10, 6, 2.3),
 (10, 6, 2.6999999999999997),
 (10, 8, 0.3),
 (10, 8, 0.7),
 (10, 8, 1.0999999999999999),
 (10, 8, 1.5),
 (10, 8, 1.9),
 (10, 8, 2.3),
 (10, 8, 2.6999999999999997),
 (35, 2, 0.3),
 (35, 2, 0.7),
 (35, 2, 1.0999999999999999),
 (35, 2, 1.5),
 (35, 2, 1.9),
 (35, 2, 2.3),
 (35, 2, 2.6999999999999997),
 (35, 4, 0.3),
 (35, 4, 0.7),
 (35, 4, 1.0999999999999999),
 (35, 4, 1.5),
 (35, 4, 1.9),
 (35, 4, 2.3),
 (35, 4, 2.6999999999999997),
 (35, 6, 0.3),
 (35, 6, 0.7),
 (35, 6, 1.0999999999999999),
 (35, 6, 1.5),
 (35, 6, 1.9),
 (35, 6, 2.3),
 (35, 6, 2.6999999999999997),
 (35, 8, 0.3),
 (35, 8, 0.7),
 (35, 8, 1.0999999999999999),
 (35, 8, 1

In [18]:
#np.where(data_labels_estim[:,None]==np.array(classnames_)[None,:])[1].shape

In [19]:
from tqdm import tqdm 
import math
from sklearn.metrics import accuracy_score

max_score = -math.inf
best_config = {"epochs":0,"t_pahse1":0,"t_phase2":0}
combinations = itertools.product(epochs,tmp_phase1,tmp_phase2)
for num_epochs,t_pahse1,t_phase2 in tqdm(combinations):
    sm.train(n_job=1, 
         verbose=None, 
         train_rough_len=num_epochs, 
         train_rough_radiusin=t_pahse1,
         train_rough_radiusfin=1.25,
         train_finetune_len=num_epochs,
         train_finetune_radiusin=t_phase2,
         train_finetune_radiusfin=0.10,
         watch_evolution = False)
    
    classnames_ = classnames
    classnames_.append('Nan')
    classnames_ = np.array(classnames_)
    #get neurone labels
    sm.node_labels_from_data(sData_train)
    neuron_labels = np.array(sm.node_labels)
    #get true labels
    classes_desirees = np.array([reverse_classnames[x] for x in data_labels_val])
    #get predicted labels
    sData_val = SOMData(data_val)
    data_labels_estim =neuron_labels[sm.find_bmu(data_val).astype(int).T[:,0]]
    classes_estimees = np.array([reverse_classnames[x] for x in data_labels_estim])
    score = accuracy_score(classes_desirees,classes_estimees)
    if score > max_score :
        max_score = score
        best_config["epochs"] = num_epochs
        best_config["t_pahse1"] = t_pahse1
        best_config["t_pahse2"] = t_phase2
        print("new best score {}".format(score))
        
print(best_config)
print(max_score)

0it [00:00, ?it/s]

Training ...
 Rough training...
radius_ini: 2.000 , radius_final: 1.250, trainlen: 10

Epoch : 0 qErr : 0.2016  tErr : 0.7428
Epoch : 1 qErr : 0.1944  tErr : 0.7341
Epoch : 2 qErr : 0.1876  tErr : 0.7110
Epoch : 3 qErr : 0.1800  tErr : 0.7197
Epoch : 4 qErr : 0.1742  tErr : 0.7168
Epoch : 5 qErr : 0.1690  tErr : 0.7139
Epoch : 6 qErr : 0.1645  tErr : 0.7168
Epoch : 7 qErr : 0.1603  tErr : 0.7486
Epoch : 8 qErr : 0.1560  tErr : 0.7572
Epoch : 9 qErr : 0.1519  tErr : 0.7572


0it [00:05, ?it/s]


KeyboardInterrupt: 

**Affichage des T-SNE**

In [ ]:
sData_test = SOMData(data_test)
sm.find_bmu(data_test).astype(int).T[:,0]

In [ ]:
sm._bmu[0].astype(int)

In [ ]:
#sm.plot_tsne()

#### Autres affichages graphiques :

- **Affichage de la carte**

In [ ]:
from mapview import View2D
dim = sData_train._dim
col_sz = 4
view2D  = View2D(16,4*(dim/col_sz),
                 "codage "+strChoix,
                 text_size=10)
view2D.show(sm, col_sz=col_sz, which_dim="all", denormalize=True) ; 

- **Affichage de la carte avec etiquettes (les indices des neurones)**

In [ ]:
view2D  = View2D(16,10,
                 "codage "+strChoix,
                 text_size=10) ;
view2D.show(sm, neuronLabels=None,anotate=True, col_sz=1, which_dim=1, denormalize=True, labelsize=16) ; 

 - **Affichage de la carte avec les cardinalités**

In [ ]:
from bmuhits import BmuHitsView
vhts  = BmuHitsView(10,10,"Hits Map",text_size=7)
vhts.show(sm, anotate=True, onlyzeros=False, labelsize=12, logaritmic=False)

 - **Affichage de la carte (distances entre les référents/neurones)**

In [ ]:
from umatrix import UMatrixView
umat = UMatrixView(16,10,"Unified Distance Matrix", text_size=20)
umat.show(sm) ;

#### Evaluer l'apprentissage de la carte

* **Calcul des erreurs globales**

In [ ]:
topographic_error = sm.calculate_topographic_error()
quantization_error = sm.calculate_quantization_error()
print ("Topographic error  = %s\nQuantization error = %s" % (topographic_error, quantization_error))

* **Coefficient de Silhouette**
(Pour cette partie on pourra voir la définition du [coefficient de Silhouette](https://fr.wikipedia.org/wiki/Silhouette_(clustering)) sur wikipedia ainsi que la documentation sur le site de scikit learn pour laquelle il y a un [tutoriel sur kmoyennes et silouhette](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html))    
En partitionnement de données (clustering), le coefficient de silhouette est une mesure de qualité d'une partition d'un ensemble de données en classification automatique. Pour chaque point, son coefficient de silhouette est la différence entre la distance moyenne avec les points du même groupe que lui (cohésion) et la distance moyenne avec les points des autres groupes voisins (séparation). Si cette différence est négative, le point est en moyenne plus proche du groupe voisin que du sien : il est donc mal classé. A l'inverse, si cette différence est positive, le point est en moyenne plus proche de son groupe que du groupe voisin : il est donc bien classé.

Le coefficient de silhouette proprement dit est la moyenne du coefficient de silhouette pour tous les points. 

In [ ]:
# sm._bmu = sm._bmu[0].astype(int)

In [ ]:
silhouette_avg = silhouette_score(sm._data, sm._bmu[0].astype(int))
silhouette_avg

In [ ]:
sample_silhouette_values = coefficientSilhouette = silhouette_samples(sData_train._data, sm._bmu[0].astype(int))
# sample_silhouette_values

#### Labellisation des neurones référents à partir des labels des données

In [ ]:
sm.node_labels_from_data(sData_train)

In [ ]:
neuron_labels = np.array(sm.node_labels)
print(neuron_labels)

 - **Affichage de la carte avec etiquettes (les votes)**

In [ ]:
view2D  = View2D(16,10,
                 "codage "+strChoix,
                 text_size=10) ;
view2D.show(sm, neuronLabels=sm.node_labels,anotate=True, col_sz=1, which_dim=1, denormalize=True,labelsize=12) ; 

#### Classification Ascendante Hiérarchique
- **Affichage du dendrogramme des référents**

In [ ]:
from dendrogram import DendrogramView
dendrogram = DendrogramView(10,10,"Dendrogramme de l'arbre hierarchique", text_size = 10)
dendrogram.show(sm)

In [ ]:
from hitmap import HitMapView
hits  = HitMapView(16,10,"Clustering",text_size=10)
hits.show??

In [ ]:
from hitmap import HitMapView
sm.cluster(10)
hits  = HitMapView(16,10,"Clustering",text_size=10)
a=hits.show(sm,labelsize=20)

#### Un peu de supervisé (pas toujours possible)
Comme les données sont associées à des classes, nous allons en profiter pour voir la correspondance entre les classes issues de la carte (déduites uniquement des entrées de la carte) et celles fournies avec les données.    
**On rappelle qu'il s'agit d'un algorithme non supervisé.**    
Ainsi, on ne prend pas en compte les classes lors de l'apprentissage.    
**On cherche donc à évaluer la capacité de la carte à discriminer ces classes. Cela ne pourra être fait que si les données des différentes classes sont bien distinctes dans l'espace des données (i.e. les entrées de la carte).** 

**Détermination des indices des classes estimées et désirées**

On ajoute la classe de rejet

In [ ]:
classnames_ = classnames
classnames_.append('Nan')
classnames_ = np.array(classnames_)
classnames_

Détermination des indices des classes désirées

In [ ]:
_,classes_desirees = np.where(data_labels_train[:,None]==np.array(classnames)[None,:])
#classes_desirees

In [ ]:
neuron_labels[sm._bmu[0].astype(int)]

Par précaution, on recalcule les etiquettes des neurones

In [ ]:
# classes esimees
data_labels_estim = neuron_labels[sm._bmu[0].astype(int)]
_, classes_estimees = np.where(data_labels_estim[:,None]==np.array(classnames)[None,:])
#classes_estimees

**Matrice de confusion**

In [ ]:
plt.figure(figsize=(10,10)) 
plot_confusion_matrix(classes_desirees, classes_estimees, classnames_,
                          title='', fontsize=12) ;

In [ ]:
#Test

In [ ]:
_,classes_desirees = np.where(data_labels_val[:,None]==np.array(classnames)[None,:])


In [ ]:
sData_val = SOMData(data_val)
data_labels_estim =neuron_labels[sm.find_bmu(data_val).astype(int).T[:,0]]
_, classes_estimees = np.where(data_labels_estim[:,None]==np.array(classnames)[None,:])

In [ ]:
plt.figure(figsize=(10,10)) 
plot_confusion_matrix(classes_desirees, classes_estimees, classnames_,
                          title='', fontsize=12) ;

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(classes_desirees,classes_estimees)